In [1]:
import numpy as np
import pandas as pd
import ssl
import os
import yaml
import shutil

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from local_glm_boost import LocalGLMBooster
from local_glm_boost.utils.tuning import tune_n_estimators
from local_glm_boost.utils.logger import LocalGLMBoostLogger

config_name = "real_data_study"

# Set up output folder, configuration file, run_id and logger
script_dir = "/home/heza7322/PycharmProjects/local-glm-boost/scripts"
folder_path = os.path.join(script_dir, "../data/results/")
config_path = os.path.join(script_dir, f"{config_name}.yaml")
with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

if os.path.exists(folder_path) and os.listdir(folder_path):
    run_id = (
        max(
            [int(folder_name.split("_")[1]) for folder_name in os.listdir(folder_path)]
            + [0]
        )
        + 1
    )
else:
    run_id = 0

output_path = os.path.join(folder_path, f"run_{run_id}")
os.makedirs(output_path)

# Put the config in the output folder
shutil.copyfile(config_path, f"{output_path}/config.yaml")

# Set up logger
logger = LocalGLMBoostLogger(
    verbose=2,
    output_path=output_path,
)
logger.append_format_level(f"run_{run_id}")

# Load stuff from the config
logger.log("Loading configuration")
n = config["n"]
features_to_use = config["features_to_use"]
target = config["target"]
weights = config["weights"]
distribution = config["distribution"]
n_estimators_max = config["n_estimators_max"]
learning_rate = config["learning_rate"]
min_samples_split = config["min_samples_split"]
min_samples_leaf = config["min_samples_leaf"]
max_depth = config["max_depth"]
glm_init = config["glm_init"]
random_seed = config["random_seed"]
n_splits = config["n_splits"]
test_size = config["test_size"]
parallel = config["parallel"]
stratified = config["stratified"]
n_jobs = config["n_jobs"]

# Load and preprocess data
logger.log("Loading data")
ssl._create_default_https_context = ssl._create_unverified_context
df_num = fetch_openml(data_id=41214, as_frame=True).data
df_sev = fetch_openml(data_id=41215, as_frame=True).data
df_sev_tot = df_sev.groupby('IDpol')['ClaimAmount'].sum()
df = df_num.merge(df_sev_tot, left_on='IDpol', right_index=True, how='left')
df.loc[df['ClaimAmount'].isna(), 'ClaimNb'] = 0
df.loc[df['ClaimAmount'].isna(), 'ClaimAmount'] = 0
df = df.loc[df['ClaimNb'] <=5]

df['Exposure'] = df['Exposure'].clip(0, 1)
df['Area'] = df['Area'].apply(lambda x: ord(x) - 65)
df['VehGas'] = df['VehGas'].apply(lambda x: 1 if x == 'Regular' else 0)

continous_features = ['VehPower','VehAge','DrivAge','BonusMalus','Density','Area','VehGas']
features = [feature for feature in continous_features if feature in features_to_use]
parallel_fit = []
for feature in ['VehBrand','Region']:
    if feature in features_to_use:
        dummies = pd.get_dummies(df[feature], prefix=feature)
        df = pd.concat([df, dummies], axis=1)
        dummy_feature_indices = [j for j in range(len(features),len(features)+len(dummies.columns))]
        parallel_fit.append(dummy_feature_indices)
        features += dummies.columns.tolist()

rng = np.random.default_rng(seed=random_seed)
if n != "all":
    df = df.sample(n, random_state = rng.integers(0, 10000))
n = len(df)

X = df[features].astype(float)
y = df[target]
w = df[weights]

X = X / X.max(axis=0)


[2023-09-18 10:58][run_41][Loading configuration]
[2023-09-18 10:58][run_41][Loading data]


In [2]:
# Fit a GLM using statsmodels using poisson deviance
import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod.families.family import Poisson

logger.log("Fitting a GLM using statsmodels")
glm = GLM(y, sm.add_constant(X), family=Poisson(), freq_weights=w)
glm_results = glm.fit()
glm_results.summary()


[2023-09-18 10:59][run_41][Fitting a GLM using statsmodels]


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                ClaimNb   No. Observations:               678007
Model:                            GLM   Df Residuals:                358319.81
Model Family:                 Poisson   Df Model:                           38
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -71121.
Date:                Mon, 18 Sep 2023   Deviance:                   1.0710e+05
Time:                        10:59:12   Pearson chi2:                 3.76e+05
No. Iterations:                     7   Pseudo R-squ. (CS):           0.006628
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -4.5989      0.051    -90.281      0.000      -4.699      -4.499
VehPower         0.5456      0.060      9.101      0.000       0.428       0.663
VehAge          -2.2187      0.154    -14.437      0.000      -2.520      -1.918
DrivAge          1.0674      0.054     19.671      0.000       0.961       1.174
BonusMalus       5.9096      0.093     63.219      0.000       5.726       6.093
Density         -0.0348      0.070     -0.500      0.617      -0.171       0.102
Area             0.4362      0.037     11.773      0.000       0.364       0.509
VehGas          -0.1717      0.015    -11.208      0.000      -0.202      -0.142
VehBrand_B1     -0.3798      0.019    -20.253      0.000      -0.417      -0.343
VehBrand_B10    -0.3960      0.043     -9.272      0.000      -0.480      -0.312
VehBrand_B11    -0.2662      0.046     -5.786      0.000      -0.356      -0.176
VehBrand_B12    -0.9011      0.024    -36.923      0.000      -0.949      -0.853
VehBrand_B13    -0.3122      0.048     -6.520      0.000      -0.406      -0.218
VehBrand_B14    -0.5915      0.096     -6.137      0.000      -0.780      -0.403
VehBrand_B2     -0.3909      0.019    -20.756      0.000      -0.428      -0.354
VehBrand_B3     -0.3500      0.026    -13.539      0.000      -0.401      -0.299
VehBrand_B4     -0.3476      0.035     -9.878      0.000      -0.417      -0.279
VehBrand_B5     -0.2954      0.030     -9.930      0.000      -0.354      -0.237
VehBrand_B6     -0.3681      0.034    -10.903      0.000      -0.434      -0.302
Region_R11      -0.2081      0.034     -6.064      0.000      -0.275      -0.141
Region_R21      -0.2791      0.141     -1.978      0.048      -0.556      -0.003
Region_R22      -0.1087      0.070     -1.562      0.118      -0.245       0.028
Region_R23      -0.3909      0.087     -4.512      0.000      -0.561      -0.221
Region_R24      -0.1947      0.023     -8.364      0.000      -0.240      -0.149
Region_R25      -0.1315      0.054     -2.422      0.015      -0.238      -0.025
Region_R26      -0.2112      0.065     -3.247      0.001      -0.339      -0.084
Region_R31      -0.2233      0.043     -5.191      0.000      -0.308      -0.139
Region_R41      -0.2732      0.054     -5.032      0.000      -0.380      -0.167
Region_R42      -0.1992      0.120     -1.653      0.098      -0.435       0.037
Region_R43      -0.4903      0.216     -2.275      0.023      -0.913      -0.068
Region_R52      -0.1364      0.034     -4.068      0.000      -0.202      -0.071
Region_R53      -0.1365      0.031     -4.405      0.000      -0.197      -0.076
Region_R54      -0.1431      0.044     -3.248      0.001      -0.230      -0.057
Region_R72      -0.1648      0.040     -4.131      0.000      -0.243      -0.087
Region_R73      -0.5051      0.065     -7.714      0.000      -0.633      -0.377
Regio

In [6]:
df['VehBrand'].unique()


['B12', 'B6', 'B3', 'B2', 'B5', ..., 'B14', 'B13', 'B4', 'B1', 'B11']
Length: 11
Categories (11, object): ['B1', 'B10', 'B11', 'B12', ..., 'B3', 'B4', 'B5', 'B6']